# Projet maison n° 3

In [1]:
# imports
import pandas as pd
import numpy as np

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [2]:
import pandas as pd
import glob

# names us
def df_names_us():
    
    global result
    
    txt_files = glob.glob("names/*.txt")

    liste = list()
    for i in range(141):
        data = pd.read_csv(txt_files[i], header = None)
        data.columns = ["name", "gender", "births"]
        data['year'] = int(txt_files[i][9:13])
        data = data.reindex(columns=['year','name','gender','births'])
        liste.append(data)

    result = pd.concat(liste, ignore_index=True)
    
    return result

df_names_us() 


,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
2020858,2020,Zykell,M,5
2020859,2020,Zylus,M,5
2020860,2020,Zymari,M,5
2020861,2020,Zyn,M,5


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [3]:
def df_names_fr():
    
    df = pd.read_csv('nat2020.csv', sep=';')
    
    df.rename(columns={'sexe': 'gender', 'preusuel': 'name', 'annais': 'year', 'nombre': 'births'}, inplace=True)
    
    df = df.reindex(columns=['year','name','gender','births'])
    df['gender'] = df['gender'].replace([1,2],['M','F'])
    df['year'] = (pd.to_numeric(df['year'],errors='coerce').fillna(np.nan))
    df['year'] = df['year'].astype('Int64')
    df['name'] = df['name'].astype(dtype = "str")
    df = df.loc[df['name'].str.len() >= 2]
    df['name'] = df['name'].str.title()
    df = df[(df['name'] != '_Prenoms_Rares') & (df["year"] !="XXXX")]
    df = df.sort_values(by=['year', 'gender', 'births', 'name'], ascending=[True, True, False, True], ignore_index=True)
    
    return df

df_names_fr()

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696
...,...,...,...,...
630402,2020,Élouen,M,3
630403,2020,Énes,M,3
630404,2020,Éphraïm,M,3
630405,2020,Éros,M,3


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [4]:
# taux de change
def df_taux_change(devises):
    df = pd.DataFrame([], columns=devises)
    return df

In [5]:
# taux de change
def df_taux_change(devises):
    df=pd.read_csv('Webstat_Export_20210929.csv',sep=";",skiprows=[1,2,3,4,5],index_col='Titre :',na_values=['-',''])
    
    df=df.dropna()
    df.columns=[col[-4:-1] for col in df.columns]
    
    df.index=pd.to_datetime(df.index,format='%d/%m/%Y')
    
    df=df.rename_axis('Date')
    df=df.replace(',','.',regex=True)
    
    df = df.astype(float)
    
    df = df.loc[:,['CHF', 'GBP', 'USD']]
    return df

### Tests

In [6]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [7]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [8]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 4.328s

OK
